In [1]:
import numpy as np
import scipy as sp


In [3]:
f = 100e3
time_interval = 1/f
tau = 0
pc = 0

In [4]:
# loran-c 300
t = np.linspace(0, 300, 30000)

envelope = (t-tau)**2*np.exp(-2*(t-tau)/65)
normalized_e = (envelope - np.min(envelope))/(np.max(envelope)-np.min(envelope))
s = normalized_e *np.sin(2*np.pi*f*t*1e-6 + pc)

In [5]:
import os
os.getcwd()

'/Users/cheolj/Desktop/coder/TIL/Signal Processing/code'

In [7]:
path = './data'
file_name = 'raw_data_0304_160Mbps_5sec.cap0'
raw_data = np.fromfile(os.path.join(path, file_name), dtype='int16')

In [8]:
normalized_rd = (raw_data - np.min(raw_data))/(np.max(raw_data) - np.min(raw_data))

In [10]:
len(raw_data)


50103518

In [11]:
raw_data = raw_data[:50000000]

In [13]:
len(raw_data)

50000000

In [9]:
import matplotlib.pyplot as plt

data_length = len(raw_data)
timespace = np.linspace(0, 5, )

fig, axes = plt.subplots(2, 1, figsize=(12, 10))


array([0.44597691, 0.44597691, 0.44233793, ..., 0.42836605, 0.42683858,
       0.42701828])